In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
CODE_DIR = '../../GPEN'
os.chdir(f'./{CODE_DIR}')

In [2]:
# enhance faces
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import __init_paths
from face_enhancement import FaceEnhancement

In [4]:
def faceEnhance(img, filename='output.jpg', sr_scale=2, sr_size=512, tile_size=64):
    model = {'name':'GPEN-BFR-512', 'sr_model':'realesrnet', 'sr_scale': int(sr_scale), 'size': int(sr_size), 'channel_multiplier':2, 'narrow':1}
    outdir = 'examples/outs-bfr'
    os.makedirs(outdir, exist_ok=True)
    faceenhancer = FaceEnhancement(in_size=model['size'], model=model['name'], use_sr=True, sr_model=model['sr_model'], sr_scale=model['sr_scale'], tile_size=int(tile_size), channel_multiplier=model['channel_multiplier'], narrow=model['narrow'])
    img_out, orig_faces, enhanced_faces = faceenhancer.process(img, aligned=False)
    img = cv2.resize(img, img_out.shape[:2][::-1])
    cv2.imwrite(os.path.join(outdir, '.'.join(filename.split('.')[:-1])+'_COMP.jpg'), np.hstack((img, img_out)))
    cv2.imwrite(os.path.join(outdir, '.'.join(filename.split('.')[:-1])+'_GPEN.jpg'), img_out)
    for m, (ef, of) in enumerate(zip(enhanced_faces, orig_faces)):
        of = cv2.resize(of, ef.shape[:2])
        cv2.imwrite(os.path.join(outdir, '.'.join(filename.split('.')[:-1])+'_face%02d'%m+'.jpg'), np.hstack((of, ef)))

    return img_out

In [5]:
import numpy as np
import gradio as gr

In [6]:
# demo = gr.Blocks()


# with demo:
#     gr.Markdown("face Enhancement.")
#     with gr.Tabs():
#         with gr.TabItem("faceEnhancement"):

#             with gr.Row():
#                 with gr.Column():
#                     image_input = gr.Image()
#                     text_input = gr.Text()
#                 image_output = gr.Image()
#             image_button = gr.Button("Run")
#     image_button.click(faceEnhance, inputs=[image_input, text_input], outputs=image_output)

# demo.launch(server_name='0.0.0.0')
        
    

In [7]:
gr.Interface(
    faceEnhance,
    inputs=[
        gr.inputs.Image(),
        gr.Text(),
        gr.Radio(['2', '4'], value='2'),
        gr.Number(value=512),
        gr.Number(value=64),
    ],
    outputs=gr.outputs.Image()
).launch(server_name='0.0.0.0')
        
    

/opt/conda/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Running on local URL:  http://localhost:7860/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7f6da73fb2b0>, 'http://localhost:7860/', None)

In [8]:
!nvidia-smi

Wed Jun 29 09:45:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:3F:00.0 Off |                  N/A |
| 39%   26C    P2   100W / 350W |   1250MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------